# TUTORIAL DE LA LIBRERIA POLARS


In [ ]:
#"instalacion"
!pip install polars
#llamando as la libreria polars
import polars as pl

In [ ]:
import polars as pl
import pandas as pd
import numpy as np
df = pl.DataFrame(
    {
        "nrs": [1, 2, 3, None, 5],
        "names": ["foo", "ham", "spam", "egg", None],
        "random": np.random.rand(5),
        "groups": ["A", "A", "B", "C", "B"],
    }
)
print(df)


In [ ]:
#EXPRESIONES:
#son funciones/métodos utilizados a la hora de realizar operaciones con datos en Polars
#(e.g., selección, creación y manipulación de columnas, aplicación de filtros, entre otros).
df = pl.DataFrame(
    {
        "nrs": [1, 2, 3, None, 5],
        "names": ["foo", "ham", "spam", "egg", None],
        "random": np.random.rand(5),
        "groups": ["A", "A", "B", "C", "B"],
    }
)
print(df)

#se realizan operaciones lógicas
df_logical = df.select(
    [
        (pl.col("nrs") > 1).alias("nrs > 1"),
        (pl.col("random") <= 0.5).alias("random < .5"),
        (pl.col("nrs") != 1).alias("nrs != 1"),
        (pl.col("nrs") == 1).alias("nrs == 1"),
        ((pl.col("random") <= 0.5) & (pl.col("nrs") > 1)).alias("and_expr"),  # and
        ((pl.col("random") <= 0.5) | (pl.col("nrs") > 1)).alias("or_expr"),  # or
    ]
)
print(df_logical)



In [ ]:
#FUNCIONES
#Las expresiones tienen una gran cantidad de funciones incorporadas. 
#Estos le permiten crear consultas complejas sin necesidad de funciones definidas por el usuario .

#-Selección de columnas
#todas las coumnas
df_all = df.select([pl.col("*")])
print(df_all)

#de todas exepto de una
df_exclude = df.select([pl.exclude("groups")])
print(df_exclude)

#Condicionales
#Polarsadmite condiciones similares a if en expresión con la sintaxis when, then, otherwise
df_conditional = df.select(
    [
        pl.col("nrs"),
        pl.when(pl.col("nrs") > 2)
        .then(pl.lit(True))
        .otherwise(pl.lit(False))
        .alias("conditional"),
    ]
)
print(df_conditional)




In [ ]:
#DATOS PERDIDOS
# Los datos que faltan se representan en Flecha y Polars con un nullvalor. 
#Este nullvalor faltante se aplica a todos los tipos de datos, incluidos los valores numéricos.
df = pl.DataFrame(
    {
        "value": [1, None],
    },
)
print(df)
null_count_df = df.null_count()
print(null_count_df)

#Relleno de datos faltantes
#os datos que faltan en un Seriesse pueden completar con el método fill_null
df = pl.DataFrame(
    {
        "col1": [1, 2, 3],
        "col2": [1, None, 3],
    },
)
print(df)
#Rellenar con el valor literal especificado
#Podemos completar los datos que faltan con un valor literal especificado con pl.lit:
fill_literal_df = (
    df.with_columns(
        pl.col("col2").fill_null(
            pl.lit(2),
        ),
    ),
)
print(fill_literal_df)

#Rellenar con una expresión
#Para mayor flexibilidad, podemos completar los datos que faltan con una expresión. 
#Por ejemplo, para llenar nulos con el valor de la mediana de esa columna

fill_median_df = df.with_columns(
    pl.col("col2").fill_null(pl.median("col2")),
)
print(fill_median_df)

#Rellenar con interpolación
#Además, podemos rellenar con interpolación (sin usar la función fill_null)
fill_interpolation_df = df.with_columns(
    pl.col("col2").interpolate(),
)
print(fill_interpolation_df)



In [ ]:
#PLIEGUES
#Polarsproporciona expresiones/métodos para agregaciones horizontales como sum, min, mean, etc. 
#estableciendo el argumento axis=1. Sin embargo, cuando necesite una agregación más compleja, 
#Polarses posible que los métodos predeterminados no sean suficientes. Ahí es cuando "fold" ses útil.
#La foldexpresión opera en columnas para máxima velocidad.

#suma manual
df = pl.DataFrame(
    {
        "a": [1, 2, 3],
        "b": [10, 20, 30],
    }
)

out = df.select(
    pl.fold(acc=pl.lit(0), function=lambda acc, x: acc + x, exprs=pl.all()).alias(
        "sum"
    ),
)
print(out)

#Condicional
#En el caso de que desee aplicar una condición/predicado en todas las columnas de una DataFrameoperación fold, 
#puede ser una forma muy concisa de expresarlo.

df = pl.DataFrame(
    {
        "a": [1, 2, 3],
        "b": [0, 1, 2],
    }
)

out = df.filter(
    pl.fold(
        acc=pl.lit(True),
        function=lambda acc, x: acc & x,
        exprs=pl.col("*") > 1,
    )
)
print(out)

In [ ]:
#cálculos sabios de filas
#Las expresiones polares funcionan sobre columnas que tienen la garantía de que están formadas por datos homogéneos. 
#Las columnas tienen esta garantía, las filas DataFrameno tanto. Por suerte tenemos un tipo de datos que tiene la garantía
#de que las filas sean homogéneas: pl.Listtipo de datos.

grades = pl.DataFrame(
    {
        "student": ["bas", "laura", "tim", "jenny"],
        "arithmetic": [10, 5, 6, 8],
        "biology": [4, 6, 2, 7],
        "geography": [8, 4, 9, 7],
    }
)
print(grades)

#ahora calculamos en una fila Si queremos calcular el valor rankde todas las columnas excepto "student"
out = grades.select([pl.concat_list(pl.all().exclude("student")).alias("all_grades")])
print(out)

#Numpy
# En polars as expresiones admiten NumPy, esto significa que si no proporciona una función Polars, 
#podemos usar NumPyy aún tenemos una operación columnar rápida a través de la NumPyAPI
import polars as pl
import numpy as np

df = pl.DataFrame({"a": [1, 2, 3], "b": [4, 5, 6]})

out = df.select(
    [
        np.log(pl.all()).suffix("_log"),
    ]
)
print(out)
